In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
import pywt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import tensorflow as tf
from keras.optimizers import SGD, Adam
from tensorflow import keras
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanSquaredError
import joblib
from tensorflow.keras.models import model_from_json


In [2]:
#doc dataset
df = pd.read_csv("ALL_5T.csv")
df.head()
print(df.isnull().sum())
#xử lí giá trị âm
colum_press = ['BO1016_Pressure(m)', 'BO1017_Pressure(m)', 'BO1021_Pressure', 'BO1022_Pressure(m)', 'BO1029_Pressure(m)', 'NB1019_Pressure', 'TD1029_Pressure', 'TD1043_Press(m)']
df[colum_press] = df[colum_press].apply(lambda x:x+5)
df.describe()
#tách tập trạng thái và tập điều khiển
df.set_index("Date_time", inplace=True) #set index
control = df["BOO_TM2_Pressure"]
# state = df
state = df.drop([ "BOO_TM2_Pressure"], axis = 1)
features = state.columns
scaler =MMS() #scale data
state_scaler = scaler.fit_transform(state[features])
state = pd.DataFrame(columns=features, data=state_scaler, index = df["BOO_TM2_Pressure"].index)
control_scaler = scaler.fit_transform(control.values.reshape(-1, 1)) 
control = pd.DataFrame(columns=["BOO_TM2_Pressure"], data=control_scaler, index = df.index)
# check
print(state.head())
print(control.head())

Date_time             0
BO1013_FLOW1          0
BO1016_Pressure(m)    0
BO1017_Pressure(m)    0
BO1021_FLOW1)         0
BO1021_Pressure       0
BO1022_flow1          0
BO1022_Pressure(m)    0
BO1017_FLOW1          0
BO1029_FLOW1          0
BO1029_Pressure(m)    0
NB1013_FLOW1          0
NB1017_FLOW1          0
NB1017_Pressure       0
NB1019_FLOW1          0
NB1019_Pressure       0
NB1020_FLOW1          0
NB1021_FLOW1          0
NB1022_FLOW1          0
TD1029_flow1          0
TD1029_Pressure       0
TD1043_FLOW1          0
TD1043_Press(m)       0
NMNBOOTM1_FLOW1       0
BOO_TM2_Pressure      0
dtype: int64
                BO1013_FLOW1  BO1016_Pressure(m)  BO1017_Pressure(m)  \
Date_time                                                              
9/15/2023 0:00      0.463406            0.616835            0.374450   
9/15/2023 0:05      0.459188            0.618765            0.379352   
9/15/2023 0:10      0.454970            0.620694            0.384254   
9/15/2023 0:15      0.45075

In [3]:
# tạo biến thử
dense_act3 = ['ReLU', 'elu', 'tanh', 'sigmoid', 'selu'] 
recur_act3 = ['ReLU', 'elu', 'tanh', 'sigmoid', 'selu']
epoch3 = [100]
# epoch = [100, 256, 500]
learning_r3 = [0.001]
# learning_r = [0.0005, 0.001, 0.002] 
ls = 'mae'
# ls = ['mae', 'mse']
'''opt1 = SGD()
opt2 = Adam()'''
print('yes')
kind = []
sc = []

yes


In [4]:
X_state = []
X_control = []
Y = []
# Loop qua các hàng của dataframe để lấy các cửa sổ thời gian t-5 đến t và giá trị tại t+1 cho Y
for i in range(8, len(df) - 1):
    X_state.append(state.iloc[i-8:i].values)  # Lấy cửa sổ thời gian state[0:8] ; control[8]
    X_control.append(control.iloc[i-8:i].values)
    Y.append(control.iloc[i])  # Lấy giá trị t+1 của cột cuối cùng
X_state = np.array(X_state)
X_control = np.array(X_control)
Y = np.array(Y)
X_state.shape, X_control.shape, Y.shape


((31381, 8, 23), (31381, 8, 1), (31381, 1))

In [5]:
X_train1, y_train1 = X_state[:20000], Y[:20000]
X_train2, y_train2 = X_control[:20000], Y[:20000]
# //////////////////////////////////////////////////
X_te1, y_te1 = X_state[20000:], Y[20000:]
X_te2, y_te2 = X_control[20000:], Y[20000:]
X_te1.shape, y_te1.shape, X_te2.shape, y_te2.shape

((11381, 8, 23), (11381, 1), (11381, 8, 1), (11381, 1))

In [12]:
for i in dense_act3:
    for j in recur_act3:
        for m in epoch3:
            for n in learning_r3:
                opt = SGD(learning_rate= n)
                model3 = Sequential([
                         tf.keras.layers.LSTM(32, recurrent_activation = j, input_shape = [8, 23]),
                         tf.keras.layers.Dense(16, activation = i),
                         tf.keras.layers.Dropout(0.05),
                         tf.keras.layers.Dense(8, activation = i),
                         tf.keras.layers.Dense(1)
                         ])
                model3.compile(optimizer = opt, loss = ls, )
                model3.fit(X_train1, y_train1, epochs= m, batch_size =1024, validation_split = 0.25)
                name = "model: "+ "dense_act3 "+ i+ " recur_act3 "+ j + " epoch3: " + str(m) +" ls "+ str(n)
                score = model3.evaluate(X_te1, y_te1)
                kind.append(name)
#                 kind.append(score)
                sc.append(score)
a = np.min(sc)
b = sc.index(a)
print("loss_min "+str(a))
print(kind[b])

Epoch 1/100
15/15 [==============================] - 3s 68ms/step - loss: 0.2190 - val_loss: 0.1502
Epoch 2/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1535 - val_loss: 0.1251
Epoch 3/100
15/15 [==============================] - 0s 31ms/step - loss: 0.1323 - val_loss: 0.1199
Epoch 4/100
15/15 [==============================] - 0s 32ms/step - loss: 0.1223 - val_loss: 0.1171
Epoch 5/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1158 - val_loss: 0.1164
Epoch 6/100
15/15 [==============================] - 0s 33ms/step - loss: 0.1117 - val_loss: 0.1168
Epoch 7/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1092 - val_loss: 0.1173
Epoch 8/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1068 - val_loss: 0.1177
Epoch 9/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1052 - val_loss: 0.1180
Epoch 10/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1027 - val_loss: 0.1184

15/15 [==============================] - 0s 27ms/step - loss: 0.0954 - val_loss: 0.0994
Epoch 63/100
15/15 [==============================] - 0s 27ms/step - loss: 0.0950 - val_loss: 0.0993
Epoch 64/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0950 - val_loss: 0.0985
Epoch 65/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0941 - val_loss: 0.0980
Epoch 66/100
15/15 [==============================] - 0s 28ms/step - loss: 0.0937 - val_loss: 0.0979
Epoch 67/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0941 - val_loss: 0.0970
Epoch 68/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0928 - val_loss: 0.0964
Epoch 69/100
15/15 [==============================] - 0s 27ms/step - loss: 0.0917 - val_loss: 0.0964
Epoch 70/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0918 - val_loss: 0.0955
Epoch 71/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0910 - val_loss: 0.0954
Epo

15/15 [==============================] - 0s 25ms/step - loss: 0.1286 - val_loss: 0.1194
Epoch 23/100
15/15 [==============================] - 0s 24ms/step - loss: 0.1281 - val_loss: 0.1193
Epoch 24/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1276 - val_loss: 0.1192
Epoch 25/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1271 - val_loss: 0.1188
Epoch 26/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1277 - val_loss: 0.1186
Epoch 27/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1268 - val_loss: 0.1185
Epoch 28/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1263 - val_loss: 0.1183
Epoch 29/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1259 - val_loss: 0.1180
Epoch 30/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1266 - val_loss: 0.1178
Epoch 31/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1260 - val_loss: 0.1176
Epo

15/15 [==============================] - 0s 27ms/step - loss: 0.0749 - val_loss: 0.0971
Epoch 84/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0748 - val_loss: 0.0965
Epoch 85/100
15/15 [==============================] - 0s 29ms/step - loss: 0.0748 - val_loss: 0.0967
Epoch 86/100
15/15 [==============================] - 0s 29ms/step - loss: 0.0748 - val_loss: 0.0968
Epoch 87/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0748 - val_loss: 0.0959
Epoch 88/100
15/15 [==============================] - 0s 27ms/step - loss: 0.0745 - val_loss: 0.0966
Epoch 89/100
15/15 [==============================] - 0s 27ms/step - loss: 0.0746 - val_loss: 0.0964
Epoch 90/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0745 - val_loss: 0.0965
Epoch 91/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0739 - val_loss: 0.0958
Epoch 92/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0748 - val_loss: 0.0965
Epo

15/15 [==============================] - 0s 27ms/step - loss: 0.1065 - val_loss: 0.0978
Epoch 44/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1053 - val_loss: 0.0974
Epoch 45/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1048 - val_loss: 0.0961
Epoch 46/100
15/15 [==============================] - 0s 32ms/step - loss: 0.1037 - val_loss: 0.0953
Epoch 47/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1031 - val_loss: 0.0951
Epoch 48/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1024 - val_loss: 0.0939
Epoch 49/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1019 - val_loss: 0.0939
Epoch 50/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1004 - val_loss: 0.0930
Epoch 51/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1001 - val_loss: 0.0926
Epoch 52/100
15/15 [==============================] - 0s 28ms/step - loss: 0.0988 - val_loss: 0.0918
Epo

...

15/15 [==============================] - 0s 28ms/step - loss: 0.1586 - val_loss: 0.1511
Epoch 67/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1589 - val_loss: 0.1511
Epoch 68/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1586 - val_loss: 0.1510
Epoch 69/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1578 - val_loss: 0.1509
Epoch 70/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1583 - val_loss: 0.1508
Epoch 71/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1574 - val_loss: 0.1507
Epoch 72/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1582 - val_loss: 0.1507
Epoch 73/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1581 - val_loss: 0.1506
Epoch 74/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1575 - val_loss: 0.1505
Epoch 75/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1577 - val_loss: 0.1505
Epo

15/15 [==============================] - 0s 28ms/step - loss: 0.1468 - val_loss: 0.1315
Epoch 27/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1461 - val_loss: 0.1314
Epoch 28/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1463 - val_loss: 0.1313
Epoch 29/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1459 - val_loss: 0.1311
Epoch 30/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1460 - val_loss: 0.1310
Epoch 31/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1465 - val_loss: 0.1309
Epoch 32/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1456 - val_loss: 0.1308
Epoch 33/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1457 - val_loss: 0.1308
Epoch 34/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1455 - val_loss: 0.1306
Epoch 35/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1454 - val_loss: 0.1305
Epo

15/15 [==============================] - 0s 27ms/step - loss: 0.0692 - val_loss: 0.0869
Epoch 88/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0691 - val_loss: 0.0921
Epoch 89/100
15/15 [==============================] - 0s 28ms/step - loss: 0.0690 - val_loss: 0.0876
Epoch 90/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0693 - val_loss: 0.0870
Epoch 91/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0694 - val_loss: 0.0874
Epoch 92/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0691 - val_loss: 0.0891
Epoch 93/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0689 - val_loss: 0.0857
Epoch 94/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0690 - val_loss: 0.0865
Epoch 95/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0686 - val_loss: 0.0872
Epoch 96/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0689 - val_loss: 0.0855
Epo

15/15 [==============================] - 0s 24ms/step - loss: 0.0959 - val_loss: 0.1126
Epoch 48/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0954 - val_loss: 0.1120
Epoch 49/100
15/15 [==============================] - 0s 23ms/step - loss: 0.0949 - val_loss: 0.1114
Epoch 50/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0948 - val_loss: 0.1106
Epoch 51/100
15/15 [==============================] - 0s 28ms/step - loss: 0.0944 - val_loss: 0.1101
Epoch 52/100
15/15 [==============================] - 0s 27ms/step - loss: 0.0929 - val_loss: 0.1098
Epoch 53/100
15/15 [==============================] - 0s 27ms/step - loss: 0.0927 - val_loss: 0.1099
Epoch 54/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0922 - val_loss: 0.1089
Epoch 55/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0916 - val_loss: 0.1086
Epoch 56/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0910 - val_loss: 0.1080
Epo

15/15 [==============================] - 0s 28ms/step - loss: 0.1149 - val_loss: 0.1336
Epoch 8/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1110 - val_loss: 0.1305
Epoch 9/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1088 - val_loss: 0.1230
Epoch 10/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1060 - val_loss: 0.1194
Epoch 11/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1030 - val_loss: 0.1151
Epoch 12/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1008 - val_loss: 0.1099
Epoch 13/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0991 - val_loss: 0.1059
Epoch 14/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0976 - val_loss: 0.1024
Epoch 15/100
15/15 [==============================] - 0s 26ms/step - loss: 0.0958 - val_loss: 0.0989
Epoch 16/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0942 - val_loss: 0.0953
Epoch

In [9]:
'''LSTM thứ 1 cho state-control'''
dense_act4 = 'tanh' 
recur_act4 = 'ReLU' 
epoch4 = 1000 
learning_r4 = 0.0005
ls = 'mae' 
opt = SGD(learning_rate= learning_r4)
model4 = Sequential([
                         tf.keras.layers.LSTM(64, recurrent_activation = 'ReLU', input_shape = [8, 23]),
                         tf.keras.layers.Dropout(0.1),
                         tf.keras.layers.Dense(32, activation = 'tanh'),
                         tf.keras.layers.Dropout(0.1),
                         tf.keras.layers.Dense(16, activation = 'tanh'),
                         tf.keras.layers.Dense(8, activation = 'tanh'),
                         tf.keras.layers.Dense(1)
                         ])
model4.compile(optimizer = opt, loss = ls, )
apply4 = model4.fit(X_train1, y_train1, epochs= epoch4, batch_size =1024, validation_split = 0.25)
score4 = model4.evaluate(X_te1, y_te1)
score4

Epoch 1/1000
15/15 [==============================] - 3s 74ms/step - loss: 0.4369 - val_loss: 0.3411
Epoch 2/1000
15/15 [==============================] - 1s 43ms/step - loss: 0.3353 - val_loss: 0.2467
Epoch 3/1000
15/15 [==============================] - 1s 48ms/step - loss: 0.2416 - val_loss: 0.1741
Epoch 4/1000
15/15 [==============================] - 1s 46ms/step - loss: 0.1855 - val_loss: 0.1503
Epoch 5/1000
15/15 [==============================] - 1s 44ms/step - loss: 0.1689 - val_loss: 0.1461
Epoch 6/1000
15/15 [==============================] - 1s 42ms/step - loss: 0.1632 - val_loss: 0.1448
Epoch 7/1000
15/15 [==============================] - 1s 42ms/step - loss: 0.1605 - val_loss: 0.1439
Epoch 8/1000
15/15 [==============================] - 1s 40ms/step - loss: 0.1594 - val_loss: 0.1429
Epoch 9/1000
15/15 [==============================] - 1s 41ms/step - loss: 0.1566 - val_loss: 0.1418
Epoch 10/1000
15/15 [==============================] - 1s 41ms/step - loss: 0.1558 - val_lo

Epoch 161/1000
15/15 [==============================] - 1s 46ms/step - loss: 0.0940 - val_loss: 0.0892
Epoch 162/1000
15/15 [==============================] - 1s 45ms/step - loss: 0.0937 - val_loss: 0.0891
Epoch 163/1000
15/15 [==============================] - 1s 48ms/step - loss: 0.0947 - val_loss: 0.0887
Epoch 164/1000
15/15 [==============================] - 1s 46ms/step - loss: 0.0943 - val_loss: 0.0887
Epoch 165/1000
15/15 [==============================] - 1s 45ms/step - loss: 0.0941 - val_loss: 0.0882
Epoch 166/1000
15/15 [==============================] - 1s 46ms/step - loss: 0.0932 - val_loss: 0.0881
Epoch 167/1000
15/15 [==============================] - 1s 46ms/step - loss: 0.0943 - val_loss: 0.0880
Epoch 168/1000
15/15 [==============================] - 1s 52ms/step - loss: 0.0936 - val_loss: 0.0881
Epoch 169/1000
15/15 [==============================] - 1s 49ms/step - loss: 0.0931 - val_loss: 0.0875
Epoch 170/1000
15/15 [==============================] - 1s 48ms/step - lo

15/15 [==============================] - 1s 47ms/step - loss: 0.0810 - val_loss: 0.0766
Epoch 320/1000
15/15 [==============================] - 1s 66ms/step - loss: 0.0809 - val_loss: 0.0765
Epoch 321/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0821 - val_loss: 0.0766
Epoch 322/1000
15/15 [==============================] - 1s 49ms/step - loss: 0.0808 - val_loss: 0.0766
Epoch 323/1000
15/15 [==============================] - 1s 54ms/step - loss: 0.0814 - val_loss: 0.0765
Epoch 324/1000
15/15 [==============================] - 1s 58ms/step - loss: 0.0811 - val_loss: 0.0765
Epoch 325/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0804 - val_loss: 0.0765
Epoch 326/1000
15/15 [==============================] - 1s 51ms/step - loss: 0.0806 - val_loss: 0.0762
Epoch 327/1000
15/15 [==============================] - 1s 60ms/step - loss: 0.0808 - val_loss: 0.0763
Epoch 328/1000
15/15 [==============================] - 1s 56ms/step - loss: 0.0810 - va

15/15 [==============================] - 1s 61ms/step - loss: 0.0768 - val_loss: 0.0718
Epoch 478/1000
15/15 [==============================] - 1s 54ms/step - loss: 0.0770 - val_loss: 0.0717
Epoch 479/1000
15/15 [==============================] - 1s 51ms/step - loss: 0.0767 - val_loss: 0.0716
Epoch 480/1000
15/15 [==============================] - 1s 57ms/step - loss: 0.0770 - val_loss: 0.0715
Epoch 481/1000
15/15 [==============================] - 1s 66ms/step - loss: 0.0775 - val_loss: 0.0715
Epoch 482/1000
15/15 [==============================] - 1s 72ms/step - loss: 0.0774 - val_loss: 0.0716
Epoch 483/1000
15/15 [==============================] - 1s 55ms/step - loss: 0.0774 - val_loss: 0.0715
Epoch 484/1000
15/15 [==============================] - 1s 59ms/step - loss: 0.0769 - val_loss: 0.0714
Epoch 485/1000
15/15 [==============================] - 1s 76ms/step - loss: 0.0771 - val_loss: 0.0713
Epoch 486/1000
15/15 [==============================] - 1s 74ms/step - loss: 0.0767 - va

15/15 [==============================] - 1s 69ms/step - loss: 0.0752 - val_loss: 0.0696
Epoch 636/1000
15/15 [==============================] - 1s 61ms/step - loss: 0.0748 - val_loss: 0.0696
Epoch 637/1000
15/15 [==============================] - 1s 53ms/step - loss: 0.0744 - val_loss: 0.0696
Epoch 638/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0756 - val_loss: 0.0697
Epoch 639/1000
15/15 [==============================] - 1s 58ms/step - loss: 0.0749 - val_loss: 0.0695
Epoch 640/1000
15/15 [==============================] - 1s 54ms/step - loss: 0.0752 - val_loss: 0.0695
Epoch 641/1000
15/15 [==============================] - 1s 56ms/step - loss: 0.0751 - val_loss: 0.0694
Epoch 642/1000
15/15 [==============================] - 1s 54ms/step - loss: 0.0760 - val_loss: 0.0695
Epoch 643/1000
15/15 [==============================] - 1s 55ms/step - loss: 0.0751 - val_loss: 0.0697
Epoch 644/1000
15/15 [==============================] - 1s 51ms/step - loss: 0.0755 - va

15/15 [==============================] - 1s 50ms/step - loss: 0.0734 - val_loss: 0.0691
Epoch 794/1000
15/15 [==============================] - 1s 51ms/step - loss: 0.0741 - val_loss: 0.0690
Epoch 795/1000
15/15 [==============================] - 1s 57ms/step - loss: 0.0732 - val_loss: 0.0691
Epoch 796/1000
15/15 [==============================] - 1s 54ms/step - loss: 0.0739 - val_loss: 0.0690
Epoch 797/1000
15/15 [==============================] - 1s 60ms/step - loss: 0.0744 - val_loss: 0.0691
Epoch 798/1000
15/15 [==============================] - 1s 58ms/step - loss: 0.0744 - val_loss: 0.0690
Epoch 799/1000
15/15 [==============================] - 1s 60ms/step - loss: 0.0741 - val_loss: 0.0689
Epoch 800/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0739 - val_loss: 0.0689
Epoch 801/1000
15/15 [==============================] - 1s 59ms/step - loss: 0.0742 - val_loss: 0.0690
Epoch 802/1000
15/15 [==============================] - 1s 50ms/step - loss: 0.0735 - va

15/15 [==============================] - 1s 56ms/step - loss: 0.0730 - val_loss: 0.0688
Epoch 952/1000
15/15 [==============================] - 1s 49ms/step - loss: 0.0731 - val_loss: 0.0690
Epoch 953/1000
15/15 [==============================] - 1s 44ms/step - loss: 0.0726 - val_loss: 0.0692
Epoch 954/1000
15/15 [==============================] - 1s 46ms/step - loss: 0.0730 - val_loss: 0.0688
Epoch 955/1000
15/15 [==============================] - 1s 48ms/step - loss: 0.0729 - val_loss: 0.0688
Epoch 956/1000
15/15 [==============================] - 1s 56ms/step - loss: 0.0724 - val_loss: 0.0689
Epoch 957/1000
15/15 [==============================] - 1s 51ms/step - loss: 0.0727 - val_loss: 0.0688
Epoch 958/1000
15/15 [==============================] - 1s 51ms/step - loss: 0.0725 - val_loss: 0.0689
Epoch 959/1000
15/15 [==============================] - 1s 48ms/step - loss: 0.0726 - val_loss: 0.0690
Epoch 960/1000
15/15 [==============================] - 1s 52ms/step - loss: 0.0730 - va

0.06847381591796875

In [6]:
'''so sánh với encoder - decoder'''
# Các tham số
dense_act4 = 'tanh'
recur_act4 = 'ReLU'
epoch4 = 1000
learning_r4 = 0.0005
ls = 'mae'
opt = SGD(learning_rate=learning_r4)

# Encoder
n_steps_in, n_features = 8, 23  # 8 bước thời gian, 23 đặc trưng
encoder_inputs = tf.keras.layers.Input(shape=(n_steps_in, n_features))
encoder_lstm = tf.keras.layers.LSTM(64, activation='tanh', recurrent_activation=recur_act4, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
n_steps_out = 1  # Chúng ta dự đoán một giá trị đầu ra
decoder_inputs = tf.keras.layers.RepeatVector(n_steps_out)(state_h)
decoder_lstm = tf.keras.layers.LSTM(64, activation='tanh', recurrent_activation=recur_act4, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(32, activation=dense_act4))(decoder_outputs)
decoder_outputs = tf.keras.layers.Dropout(0.1)(decoder_outputs)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(16, activation=dense_act4))(decoder_outputs)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(8, activation=dense_act4))(decoder_outputs)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder_outputs)

# Model
model = tf.keras.models.Model(encoder_inputs, decoder_outputs)
model.compile(optimizer=opt, loss=ls)

# Huấn luyện mô hình
apply4 = model.fit(X_train1, y_train1, epochs=epoch4, batch_size=1024, validation_split=0.25)

# Đánh giá mô hình
score4 = model.evaluate(X_te1, y_te1)

# In ra kết quả
print(f'Model Score: {score4}')

Epoch 1/1000
15/15 [==============================] - 8s 169ms/step - loss: 0.2962 - val_loss: 0.2432
Epoch 2/1000
15/15 [==============================] - 1s 80ms/step - loss: 0.2456 - val_loss: 0.1966
Epoch 3/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.1987 - val_loss: 0.1625
Epoch 4/1000
15/15 [==============================] - 1s 70ms/step - loss: 0.1697 - val_loss: 0.1523
Epoch 5/1000
15/15 [==============================] - 1s 61ms/step - loss: 0.1591 - val_loss: 0.1508
Epoch 6/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.1559 - val_loss: 0.1506
Epoch 7/1000
15/15 [==============================] - 1s 75ms/step - loss: 0.1545 - val_loss: 0.1508
Epoch 8/1000
15/15 [==============================] - 1s 67ms/step - loss: 0.1539 - val_loss: 0.1511
Epoch 9/1000
15/15 [==============================] - 1s 57ms/step - loss: 0.1534 - val_loss: 0.1513
Epoch 10/1000
15/15 [==============================] - 1s 65ms/step - loss: 0.1531 - val_l

15/15 [==============================] - 2s 117ms/step - loss: 0.1371 - val_loss: 0.1378
Epoch 82/1000
15/15 [==============================] - 1s 91ms/step - loss: 0.1370 - val_loss: 0.1377
Epoch 83/1000
15/15 [==============================] - 2s 107ms/step - loss: 0.1366 - val_loss: 0.1374
Epoch 84/1000
15/15 [==============================] - 1s 91ms/step - loss: 0.1364 - val_loss: 0.1372
Epoch 85/1000
15/15 [==============================] - 1s 89ms/step - loss: 0.1363 - val_loss: 0.1370
Epoch 86/1000
15/15 [==============================] - 1s 100ms/step - loss: 0.1362 - val_loss: 0.1368
Epoch 87/1000
15/15 [==============================] - 2s 104ms/step - loss: 0.1357 - val_loss: 0.1365
Epoch 88/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.1355 - val_loss: 0.1363
Epoch 89/1000
15/15 [==============================] - 1s 91ms/step - loss: 0.1351 - val_loss: 0.1361
Epoch 90/1000
15/15 [==============================] - 1s 77ms/step - loss: 0.1352 - val_los

15/15 [==============================] - 1s 87ms/step - loss: 0.1216 - val_loss: 0.1252
Epoch 161/1000
15/15 [==============================] - 1s 102ms/step - loss: 0.1213 - val_loss: 0.1250
Epoch 162/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.1213 - val_loss: 0.1249
Epoch 163/1000
15/15 [==============================] - 1s 80ms/step - loss: 0.1209 - val_loss: 0.1247
Epoch 164/1000
15/15 [==============================] - 2s 114ms/step - loss: 0.1207 - val_loss: 0.1246
Epoch 165/1000
15/15 [==============================] - 1s 89ms/step - loss: 0.1207 - val_loss: 0.1245
Epoch 166/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.1203 - val_loss: 0.1244
Epoch 167/1000
15/15 [==============================] - 2s 108ms/step - loss: 0.1202 - val_loss: 0.1242
Epoch 168/1000
15/15 [==============================] - 1s 90ms/step - loss: 0.1198 - val_loss: 0.1240
Epoch 169/1000
15/15 [==============================] - 1s 75ms/step - loss: 0.1198 -

15/15 [==============================] - 1s 68ms/step - loss: 0.1002 - val_loss: 0.1147
Epoch 240/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.1001 - val_loss: 0.1140
Epoch 241/1000
15/15 [==============================] - 1s 80ms/step - loss: 0.0992 - val_loss: 0.1143
Epoch 242/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.0991 - val_loss: 0.1149
Epoch 243/1000
15/15 [==============================] - 1s 78ms/step - loss: 0.0987 - val_loss: 0.1145
Epoch 244/1000
15/15 [==============================] - 1s 78ms/step - loss: 0.0981 - val_loss: 0.1140
Epoch 245/1000
15/15 [==============================] - 1s 93ms/step - loss: 0.0981 - val_loss: 0.1144
Epoch 246/1000
15/15 [==============================] - 1s 91ms/step - loss: 0.0973 - val_loss: 0.1138
Epoch 247/1000
15/15 [==============================] - 1s 98ms/step - loss: 0.0973 - val_loss: 0.1143
Epoch 248/1000
15/15 [==============================] - 1s 70ms/step - loss: 0.0969 - va

15/15 [==============================] - 1s 75ms/step - loss: 0.0751 - val_loss: 0.1159
Epoch 319/1000
15/15 [==============================] - 1s 67ms/step - loss: 0.0751 - val_loss: 0.1165
Epoch 320/1000
15/15 [==============================] - 1s 76ms/step - loss: 0.0746 - val_loss: 0.1166
Epoch 321/1000
15/15 [==============================] - 2s 101ms/step - loss: 0.0747 - val_loss: 0.1173
Epoch 322/1000
15/15 [==============================] - 1s 79ms/step - loss: 0.0744 - val_loss: 0.1161
Epoch 323/1000
15/15 [==============================] - 1s 71ms/step - loss: 0.0742 - val_loss: 0.1158
Epoch 324/1000
15/15 [==============================] - 1s 77ms/step - loss: 0.0739 - val_loss: 0.1169
Epoch 325/1000
15/15 [==============================] - 1s 74ms/step - loss: 0.0740 - val_loss: 0.1169
Epoch 326/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0736 - val_loss: 0.1178
Epoch 327/1000
15/15 [==============================] - 1s 74ms/step - loss: 0.0737 - v

...

15/15 [==============================] - 1s 64ms/step - loss: 0.0658 - val_loss: 0.1351
Epoch 635/1000
15/15 [==============================] - 1s 64ms/step - loss: 0.0658 - val_loss: 0.1350
Epoch 636/1000
15/15 [==============================] - 1s 75ms/step - loss: 0.0657 - val_loss: 0.1367
Epoch 637/1000
15/15 [==============================] - 1s 73ms/step - loss: 0.0657 - val_loss: 0.1348
Epoch 638/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0658 - val_loss: 0.1356
Epoch 639/1000
15/15 [==============================] - 1s 64ms/step - loss: 0.0657 - val_loss: 0.1362
Epoch 640/1000
15/15 [==============================] - 1s 73ms/step - loss: 0.0657 - val_loss: 0.1357
Epoch 641/1000
15/15 [==============================] - 1s 72ms/step - loss: 0.0660 - val_loss: 0.1355
Epoch 642/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0661 - val_loss: 0.1361
Epoch 643/1000
15/15 [==============================] - 1s 66ms/step - loss: 0.0656 - va

15/15 [==============================] - 1s 79ms/step - loss: 0.0653 - val_loss: 0.1362
Epoch 714/1000
15/15 [==============================] - 1s 90ms/step - loss: 0.0654 - val_loss: 0.1358
Epoch 715/1000
15/15 [==============================] - 1s 79ms/step - loss: 0.0657 - val_loss: 0.1357
Epoch 716/1000
15/15 [==============================] - 1s 81ms/step - loss: 0.0655 - val_loss: 0.1357
Epoch 717/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0653 - val_loss: 0.1354
Epoch 718/1000
15/15 [==============================] - 1s 66ms/step - loss: 0.0656 - val_loss: 0.1370
Epoch 719/1000
15/15 [==============================] - 1s 78ms/step - loss: 0.0655 - val_loss: 0.1358
Epoch 720/1000
15/15 [==============================] - 1s 75ms/step - loss: 0.0653 - val_loss: 0.1354
Epoch 721/1000
15/15 [==============================] - 1s 67ms/step - loss: 0.0655 - val_loss: 0.1357
Epoch 722/1000
15/15 [==============================] - 1s 67ms/step - loss: 0.0655 - va

15/15 [==============================] - 1s 76ms/step - loss: 0.0652 - val_loss: 0.1361
Epoch 793/1000
15/15 [==============================] - 2s 109ms/step - loss: 0.0650 - val_loss: 0.1359
Epoch 794/1000
15/15 [==============================] - 2s 121ms/step - loss: 0.0648 - val_loss: 0.1362
Epoch 795/1000
15/15 [==============================] - 1s 95ms/step - loss: 0.0649 - val_loss: 0.1364
Epoch 796/1000
15/15 [==============================] - 2s 136ms/step - loss: 0.0651 - val_loss: 0.1366
Epoch 797/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.0650 - val_loss: 0.1364
Epoch 798/1000
15/15 [==============================] - 2s 113ms/step - loss: 0.0649 - val_loss: 0.1364
Epoch 799/1000
15/15 [==============================] - 1s 89ms/step - loss: 0.0648 - val_loss: 0.1352
Epoch 800/1000
15/15 [==============================] - 1s 93ms/step - loss: 0.0648 - val_loss: 0.1365
Epoch 801/1000
15/15 [==============================] - 2s 103ms/step - loss: 0.0650

15/15 [==============================] - 2s 111ms/step - loss: 0.0642 - val_loss: 0.1356
Epoch 872/1000
15/15 [==============================] - 2s 125ms/step - loss: 0.0645 - val_loss: 0.1364
Epoch 873/1000
15/15 [==============================] - 1s 101ms/step - loss: 0.0644 - val_loss: 0.1356
Epoch 874/1000
15/15 [==============================] - 2s 107ms/step - loss: 0.0643 - val_loss: 0.1361
Epoch 875/1000
15/15 [==============================] - 2s 113ms/step - loss: 0.0644 - val_loss: 0.1371
Epoch 876/1000
15/15 [==============================] - 1s 94ms/step - loss: 0.0643 - val_loss: 0.1362
Epoch 877/1000
15/15 [==============================] - 2s 115ms/step - loss: 0.0644 - val_loss: 0.1363
Epoch 878/1000
15/15 [==============================] - 1s 82ms/step - loss: 0.0644 - val_loss: 0.1355
Epoch 879/1000
15/15 [==============================] - 1s 71ms/step - loss: 0.0645 - val_loss: 0.1368
Epoch 880/1000
15/15 [==============================] - 2s 107ms/step - loss: 0.06

15/15 [==============================] - 1s 97ms/step - loss: 0.0638 - val_loss: 0.1360
Epoch 951/1000
15/15 [==============================] - 2s 109ms/step - loss: 0.0639 - val_loss: 0.1371
Epoch 952/1000
15/15 [==============================] - 2s 109ms/step - loss: 0.0639 - val_loss: 0.1374
Epoch 953/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.0637 - val_loss: 0.1378
Epoch 954/1000
15/15 [==============================] - 2s 117ms/step - loss: 0.0639 - val_loss: 0.1370
Epoch 955/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.0640 - val_loss: 0.1377
Epoch 956/1000
15/15 [==============================] - 2s 112ms/step - loss: 0.0638 - val_loss: 0.1380
Epoch 957/1000
15/15 [==============================] - 1s 100ms/step - loss: 0.0640 - val_loss: 0.1380
Epoch 958/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.0639 - val_loss: 0.1372
Epoch 959/1000
15/15 [==============================] - 2s 123ms/step - loss: 0.063

In [12]:
# check qua thông tin model (sử dụng cho lớp LSTM thứ nhất y_trạng thái và u_control)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 8, 23)]              0         []                            
                                                                                                  
 lstm_3 (LSTM)               [(None, 64),                 22528     ['input_2[0][0]']             
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 repeat_vector_1 (RepeatVec  (None, 1, 64)                0         ['lstm_3[0][1]']              
 tor)                                                                                       

In [8]:
for i in dense_act3:
    for j in recur_act3:
        for m in epoch3:
            for n in learning_r3:
                opt = SGD(learning_rate= n)
                model3 = Sequential([
                         tf.keras.layers.LSTM(32, recurrent_activation = j, input_shape = [8, 1]),
                         tf.keras.layers.Dense(16, activation = i),
                         tf.keras.layers.Dropout(0.05),
                         tf.keras.layers.Dense(8, activation = i),
                         tf.keras.layers.Dense(1)
                         ])
                model3.compile(optimizer = opt, loss = ls, )
                model3.fit(X_train2, y_train2, epochs= m, batch_size =1024, validation_split = 0.25)
                name = "model: "+ "dense_act3 "+ i+ " recur_act3 "+ j + " epoch3: " + str(m) +" ls "+ str(n)
                score = model3.evaluate(X_te2, y_te2)
                kind.append(name)
#                 kind.append(score)
                sc.append(score)
c = np.min(sc)
d = sc.index(c)
print("loss_min "+str(c))
print(kind[d])

Epoch 1/100
15/15 [==============================] - 4s 76ms/step - loss: 0.3098 - val_loss: 0.2719
Epoch 2/100
15/15 [==============================] - 1s 42ms/step - loss: 0.2784 - val_loss: 0.2416
Epoch 3/100
15/15 [==============================] - 1s 61ms/step - loss: 0.2471 - val_loss: 0.2125
Epoch 4/100
15/15 [==============================] - 1s 44ms/step - loss: 0.2168 - val_loss: 0.1864
Epoch 5/100
15/15 [==============================] - 1s 44ms/step - loss: 0.1912 - val_loss: 0.1690
Epoch 6/100
15/15 [==============================] - 1s 48ms/step - loss: 0.1745 - val_loss: 0.1607
Epoch 7/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1659 - val_loss: 0.1574
Epoch 8/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1618 - val_loss: 0.1562
Epoch 9/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1596 - val_loss: 0.1557
Epoch 10/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1584 - val_loss: 0.1555

15/15 [==============================] - 0s 25ms/step - loss: 0.1548 - val_loss: 0.1545
Epoch 63/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1549 - val_loss: 0.1545
Epoch 64/100
15/15 [==============================] - 0s 24ms/step - loss: 0.1548 - val_loss: 0.1545
Epoch 65/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1548 - val_loss: 0.1544
Epoch 66/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1548 - val_loss: 0.1544
Epoch 67/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1547 - val_loss: 0.1544
Epoch 68/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1546 - val_loss: 0.1544
Epoch 69/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1548 - val_loss: 0.1544
Epoch 70/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1548 - val_loss: 0.1544
Epoch 71/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1547 - val_loss: 0.1544
Epo

15/15 [==============================] - 0s 26ms/step - loss: 0.1370 - val_loss: 0.1357
Epoch 23/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1366 - val_loss: 0.1354
Epoch 24/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1365 - val_loss: 0.1351
Epoch 25/100
15/15 [==============================] - 0s 23ms/step - loss: 0.1361 - val_loss: 0.1347
Epoch 26/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1358 - val_loss: 0.1344
Epoch 27/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1355 - val_loss: 0.1341
Epoch 28/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1350 - val_loss: 0.1337
Epoch 29/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1345 - val_loss: 0.1330
Epoch 30/100
15/15 [==============================] - 1s 35ms/step - loss: 0.1339 - val_loss: 0.1324
Epoch 31/100
15/15 [==============================] - 0s 32ms/step - loss: 0.1332 - val_loss: 0.1320
Epo

15/15 [==============================] - 1s 43ms/step - loss: 0.1542 - val_loss: 0.1541
Epoch 84/100
15/15 [==============================] - 1s 36ms/step - loss: 0.1541 - val_loss: 0.1541
Epoch 85/100
15/15 [==============================] - 0s 31ms/step - loss: 0.1541 - val_loss: 0.1540
Epoch 86/100
15/15 [==============================] - 0s 32ms/step - loss: 0.1541 - val_loss: 0.1540
Epoch 87/100
15/15 [==============================] - 1s 34ms/step - loss: 0.1540 - val_loss: 0.1540
Epoch 88/100
15/15 [==============================] - 0s 33ms/step - loss: 0.1540 - val_loss: 0.1540
Epoch 89/100
15/15 [==============================] - 0s 32ms/step - loss: 0.1540 - val_loss: 0.1539
Epoch 90/100
15/15 [==============================] - 1s 36ms/step - loss: 0.1540 - val_loss: 0.1539
Epoch 91/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1540 - val_loss: 0.1539
Epoch 92/100
15/15 [==============================] - 0s 32ms/step - loss: 0.1539 - val_loss: 0.1539
Epo

15/15 [==============================] - 1s 38ms/step - loss: 0.1523 - val_loss: 0.1522
Epoch 44/100
15/15 [==============================] - 1s 42ms/step - loss: 0.1523 - val_loss: 0.1521
Epoch 45/100
15/15 [==============================] - 1s 40ms/step - loss: 0.1522 - val_loss: 0.1520
Epoch 46/100
15/15 [==============================] - 1s 40ms/step - loss: 0.1519 - val_loss: 0.1518
Epoch 47/100
15/15 [==============================] - 1s 40ms/step - loss: 0.1518 - val_loss: 0.1517
Epoch 48/100
15/15 [==============================] - 1s 41ms/step - loss: 0.1517 - val_loss: 0.1516
Epoch 49/100
15/15 [==============================] - 1s 36ms/step - loss: 0.1515 - val_loss: 0.1514
Epoch 50/100
15/15 [==============================] - 0s 33ms/step - loss: 0.1515 - val_loss: 0.1513
Epoch 51/100
15/15 [==============================] - 1s 42ms/step - loss: 0.1514 - val_loss: 0.1512
Epoch 52/100
15/15 [==============================] - 1s 41ms/step - loss: 0.1512 - val_loss: 0.1511
Epo

...

15/15 [==============================] - 1s 43ms/step - loss: 0.1633 - val_loss: 0.1580
Epoch 67/100
15/15 [==============================] - 1s 43ms/step - loss: 0.1628 - val_loss: 0.1580
Epoch 68/100
15/15 [==============================] - 1s 44ms/step - loss: 0.1630 - val_loss: 0.1581
Epoch 69/100
15/15 [==============================] - 1s 35ms/step - loss: 0.1631 - val_loss: 0.1580
Epoch 70/100
15/15 [==============================] - 0s 30ms/step - loss: 0.1632 - val_loss: 0.1580
Epoch 71/100
15/15 [==============================] - 1s 40ms/step - loss: 0.1630 - val_loss: 0.1579
Epoch 72/100
15/15 [==============================] - 1s 40ms/step - loss: 0.1631 - val_loss: 0.1580
Epoch 73/100
15/15 [==============================] - 1s 35ms/step - loss: 0.1625 - val_loss: 0.1580
Epoch 74/100
15/15 [==============================] - 1s 41ms/step - loss: 0.1626 - val_loss: 0.1580
Epoch 75/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1629 - val_loss: 0.1580
Epo

15/15 [==============================] - 0s 30ms/step - loss: 0.1641 - val_loss: 0.1579
Epoch 27/100
15/15 [==============================] - 1s 42ms/step - loss: 0.1636 - val_loss: 0.1579
Epoch 28/100
15/15 [==============================] - 1s 38ms/step - loss: 0.1643 - val_loss: 0.1579
Epoch 29/100
15/15 [==============================] - 1s 40ms/step - loss: 0.1638 - val_loss: 0.1579
Epoch 30/100
15/15 [==============================] - 1s 42ms/step - loss: 0.1635 - val_loss: 0.1578
Epoch 31/100
15/15 [==============================] - 1s 39ms/step - loss: 0.1637 - val_loss: 0.1579
Epoch 32/100
15/15 [==============================] - 1s 43ms/step - loss: 0.1636 - val_loss: 0.1579
Epoch 33/100
15/15 [==============================] - 1s 39ms/step - loss: 0.1640 - val_loss: 0.1579
Epoch 34/100
15/15 [==============================] - 1s 38ms/step - loss: 0.1633 - val_loss: 0.1579
Epoch 35/100
15/15 [==============================] - 1s 42ms/step - loss: 0.1634 - val_loss: 0.1579
Epo

15/15 [==============================] - 1s 43ms/step - loss: 0.1520 - val_loss: 0.1518
Epoch 88/100
15/15 [==============================] - 1s 37ms/step - loss: 0.1519 - val_loss: 0.1517
Epoch 89/100
15/15 [==============================] - 1s 41ms/step - loss: 0.1518 - val_loss: 0.1516
Epoch 90/100
15/15 [==============================] - 1s 41ms/step - loss: 0.1517 - val_loss: 0.1515
Epoch 91/100
15/15 [==============================] - 1s 43ms/step - loss: 0.1515 - val_loss: 0.1515
Epoch 92/100
15/15 [==============================] - 1s 39ms/step - loss: 0.1515 - val_loss: 0.1513
Epoch 93/100
15/15 [==============================] - 1s 39ms/step - loss: 0.1514 - val_loss: 0.1513
Epoch 94/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1513 - val_loss: 0.1512
Epoch 95/100
15/15 [==============================] - 0s 24ms/step - loss: 0.1512 - val_loss: 0.1511
Epoch 96/100
15/15 [==============================] - 0s 24ms/step - loss: 0.1512 - val_loss: 0.1510
Epo

15/15 [==============================] - 1s 34ms/step - loss: 0.1559 - val_loss: 0.1553
Epoch 48/100
15/15 [==============================] - 1s 39ms/step - loss: 0.1557 - val_loss: 0.1552
Epoch 49/100
15/15 [==============================] - 0s 32ms/step - loss: 0.1557 - val_loss: 0.1552
Epoch 50/100
15/15 [==============================] - 1s 35ms/step - loss: 0.1558 - val_loss: 0.1551
Epoch 51/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1558 - val_loss: 0.1551
Epoch 52/100
15/15 [==============================] - 1s 34ms/step - loss: 0.1556 - val_loss: 0.1551
Epoch 53/100
15/15 [==============================] - 0s 29ms/step - loss: 0.1557 - val_loss: 0.1551
Epoch 54/100
15/15 [==============================] - 0s 34ms/step - loss: 0.1556 - val_loss: 0.1550
Epoch 55/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1556 - val_loss: 0.1550
Epoch 56/100
15/15 [==============================] - 1s 46ms/step - loss: 0.1555 - val_loss: 0.1551
Epo

15/15 [==============================] - 1s 51ms/step - loss: 0.1578 - val_loss: 0.1564
Epoch 8/100
15/15 [==============================] - 1s 51ms/step - loss: 0.1573 - val_loss: 0.1560
Epoch 9/100
15/15 [==============================] - 1s 50ms/step - loss: 0.1570 - val_loss: 0.1556
Epoch 10/100
15/15 [==============================] - 1s 51ms/step - loss: 0.1563 - val_loss: 0.1551
Epoch 11/100
15/15 [==============================] - 1s 50ms/step - loss: 0.1558 - val_loss: 0.1544
Epoch 12/100
15/15 [==============================] - 1s 50ms/step - loss: 0.1552 - val_loss: 0.1539
Epoch 13/100
15/15 [==============================] - 1s 50ms/step - loss: 0.1545 - val_loss: 0.1534
Epoch 14/100
15/15 [==============================] - 1s 50ms/step - loss: 0.1541 - val_loss: 0.1528
Epoch 15/100
15/15 [==============================] - 1s 48ms/step - loss: 0.1534 - val_loss: 0.1523
Epoch 16/100
15/15 [==============================] - 1s 41ms/step - loss: 0.1531 - val_loss: 0.1518
Epoch

In [9]:
'''LSTM thứ 2 cho control-control'''
dense_act5 = 'selu' 
recur_act5 = 'sigmoid' 
epoch5 = 1000 
learning_r5 = 0.0005
ls = 'mae' 
opt = SGD(learning_rate= learning_r5)
model5 = Sequential([
                         tf.keras.layers.LSTM(64, recurrent_activation = 'sigmoid', input_shape = [8, 1]),
                         tf.keras.layers.Dropout(0.1),
                         tf.keras.layers.Dense(32, activation = 'selu'),
                         tf.keras.layers.Dropout(0.1),
                         tf.keras.layers.Dense(16, activation = 'selu'),
                         tf.keras.layers.Dense(8, activation = 'selu'),
                         tf.keras.layers.Dense(1)
                         ])
model5.compile(optimizer = opt, loss = ls, )
apply5 = model5.fit(X_train2, y_train2, epochs= epoch4, batch_size =1024, validation_split = 0.25)
score5 = model5.evaluate(X_te2, y_te2)
score5

Epoch 1/1000
15/15 [==============================] - 6s 150ms/step - loss: 0.1133 - val_loss: 0.0867
Epoch 2/1000
15/15 [==============================] - 1s 92ms/step - loss: 0.0948 - val_loss: 0.0820
Epoch 3/1000
15/15 [==============================] - 1s 96ms/step - loss: 0.0906 - val_loss: 0.0766
Epoch 4/1000
15/15 [==============================] - 1s 64ms/step - loss: 0.0855 - val_loss: 0.0721
Epoch 5/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.0816 - val_loss: 0.0678
Epoch 6/1000
15/15 [==============================] - 2s 109ms/step - loss: 0.0782 - val_loss: 0.0639
Epoch 7/1000
15/15 [==============================] - 1s 99ms/step - loss: 0.0729 - val_loss: 0.0603
Epoch 8/1000
15/15 [==============================] - 2s 108ms/step - loss: 0.0707 - val_loss: 0.0569
Epoch 9/1000
15/15 [==============================] - 2s 106ms/step - loss: 0.0665 - val_loss: 0.0533
Epoch 10/1000
15/15 [==============================] - 1s 71ms/step - loss: 0.0637 - va

15/15 [==============================] - 1s 81ms/step - loss: 0.0303 - val_loss: 0.0184
Epoch 82/1000
15/15 [==============================] - 1s 78ms/step - loss: 0.0307 - val_loss: 0.0185
Epoch 83/1000
15/15 [==============================] - 1s 76ms/step - loss: 0.0302 - val_loss: 0.0187
Epoch 84/1000
15/15 [==============================] - 1s 67ms/step - loss: 0.0303 - val_loss: 0.0187
Epoch 85/1000
15/15 [==============================] - 1s 75ms/step - loss: 0.0301 - val_loss: 0.0184
Epoch 86/1000
15/15 [==============================] - 1s 73ms/step - loss: 0.0303 - val_loss: 0.0187
Epoch 87/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0302 - val_loss: 0.0187
Epoch 88/1000
15/15 [==============================] - 1s 65ms/step - loss: 0.0302 - val_loss: 0.0187
Epoch 89/1000
15/15 [==============================] - 1s 72ms/step - loss: 0.0298 - val_loss: 0.0186
Epoch 90/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0299 - val_loss: 0

Epoch 161/1000
15/15 [==============================] - 1s 72ms/step - loss: 0.0275 - val_loss: 0.0187
Epoch 162/1000
15/15 [==============================] - 1s 68ms/step - loss: 0.0277 - val_loss: 0.0190
Epoch 163/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0277 - val_loss: 0.0188
Epoch 164/1000
15/15 [==============================] - 1s 72ms/step - loss: 0.0275 - val_loss: 0.0187
Epoch 165/1000
15/15 [==============================] - 1s 73ms/step - loss: 0.0274 - val_loss: 0.0189
Epoch 166/1000
15/15 [==============================] - 1s 66ms/step - loss: 0.0275 - val_loss: 0.0188
Epoch 167/1000
15/15 [==============================] - 1s 61ms/step - loss: 0.0277 - val_loss: 0.0187
Epoch 168/1000
15/15 [==============================] - 1s 71ms/step - loss: 0.0274 - val_loss: 0.0188
Epoch 169/1000
15/15 [==============================] - 1s 70ms/step - loss: 0.0271 - val_loss: 0.0190
Epoch 170/1000
15/15 [==============================] - 1s 63ms/step - lo

15/15 [==============================] - 1s 73ms/step - loss: 0.0264 - val_loss: 0.0185
Epoch 241/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0262 - val_loss: 0.0185
Epoch 242/1000
15/15 [==============================] - 1s 66ms/step - loss: 0.0263 - val_loss: 0.0185
Epoch 243/1000
15/15 [==============================] - 1s 70ms/step - loss: 0.0260 - val_loss: 0.0186
Epoch 244/1000
15/15 [==============================] - 1s 67ms/step - loss: 0.0266 - val_loss: 0.0187
Epoch 245/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0265 - val_loss: 0.0185
Epoch 246/1000
15/15 [==============================] - 1s 67ms/step - loss: 0.0261 - val_loss: 0.0185
Epoch 247/1000
15/15 [==============================] - 1s 76ms/step - loss: 0.0261 - val_loss: 0.0185
Epoch 248/1000
15/15 [==============================] - 1s 71ms/step - loss: 0.0261 - val_loss: 0.0185
Epoch 249/1000
15/15 [==============================] - 1s 59ms/step - loss: 0.0264 - va

15/15 [==============================] - 1s 82ms/step - loss: 0.0255 - val_loss: 0.0179
Epoch 320/1000
15/15 [==============================] - 1s 93ms/step - loss: 0.0250 - val_loss: 0.0181
Epoch 321/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.0253 - val_loss: 0.0179
Epoch 322/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.0251 - val_loss: 0.0181
Epoch 323/1000
15/15 [==============================] - 1s 99ms/step - loss: 0.0254 - val_loss: 0.0179
Epoch 324/1000
15/15 [==============================] - 1s 92ms/step - loss: 0.0253 - val_loss: 0.0179
Epoch 325/1000
15/15 [==============================] - 1s 96ms/step - loss: 0.0249 - val_loss: 0.0183
Epoch 326/1000
15/15 [==============================] - 1s 99ms/step - loss: 0.0250 - val_loss: 0.0179
Epoch 327/1000
15/15 [==============================] - 1s 88ms/step - loss: 0.0255 - val_loss: 0.0180
Epoch 328/1000
15/15 [==============================] - 1s 101ms/step - loss: 0.0255 - v

...

15/15 [==============================] - 1s 71ms/step - loss: 0.0226 - val_loss: 0.0164
Epoch 715/1000
15/15 [==============================] - 1s 71ms/step - loss: 0.0227 - val_loss: 0.0164
Epoch 716/1000
15/15 [==============================] - 1s 65ms/step - loss: 0.0226 - val_loss: 0.0164
Epoch 717/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0230 - val_loss: 0.0164
Epoch 718/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0228 - val_loss: 0.0164
Epoch 719/1000
15/15 [==============================] - 1s 66ms/step - loss: 0.0225 - val_loss: 0.0165
Epoch 720/1000
15/15 [==============================] - 1s 61ms/step - loss: 0.0224 - val_loss: 0.0164
Epoch 721/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0226 - val_loss: 0.0164
Epoch 722/1000
15/15 [==============================] - 1s 76ms/step - loss: 0.0224 - val_loss: 0.0164
Epoch 723/1000
15/15 [==============================] - 1s 71ms/step - loss: 0.0226 - va

15/15 [==============================] - 1s 72ms/step - loss: 0.0224 - val_loss: 0.0162
Epoch 794/1000
15/15 [==============================] - 1s 74ms/step - loss: 0.0222 - val_loss: 0.0162
Epoch 795/1000
15/15 [==============================] - 1s 72ms/step - loss: 0.0223 - val_loss: 0.0162
Epoch 796/1000
15/15 [==============================] - 1s 74ms/step - loss: 0.0222 - val_loss: 0.0162
Epoch 797/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.0224 - val_loss: 0.0162
Epoch 798/1000
15/15 [==============================] - 2s 117ms/step - loss: 0.0224 - val_loss: 0.0162
Epoch 799/1000
15/15 [==============================] - 2s 102ms/step - loss: 0.0223 - val_loss: 0.0162
Epoch 800/1000
15/15 [==============================] - 1s 70ms/step - loss: 0.0222 - val_loss: 0.0162
Epoch 801/1000
15/15 [==============================] - 1s 81ms/step - loss: 0.0225 - val_loss: 0.0162
Epoch 802/1000
15/15 [==============================] - 1s 79ms/step - loss: 0.0224 - 

15/15 [==============================] - 1s 94ms/step - loss: 0.0223 - val_loss: 0.0161
Epoch 873/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.0217 - val_loss: 0.0160
Epoch 874/1000
15/15 [==============================] - 1s 81ms/step - loss: 0.0220 - val_loss: 0.0159
Epoch 875/1000
15/15 [==============================] - 1s 93ms/step - loss: 0.0218 - val_loss: 0.0159
Epoch 876/1000
15/15 [==============================] - 1s 78ms/step - loss: 0.0223 - val_loss: 0.0159
Epoch 877/1000
15/15 [==============================] - 1s 77ms/step - loss: 0.0219 - val_loss: 0.0160
Epoch 878/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.0221 - val_loss: 0.0160
Epoch 879/1000
15/15 [==============================] - 1s 81ms/step - loss: 0.0219 - val_loss: 0.0159
Epoch 880/1000
15/15 [==============================] - 1s 72ms/step - loss: 0.0224 - val_loss: 0.0159
Epoch 881/1000
15/15 [==============================] - 2s 113ms/step - loss: 0.0219 - v

15/15 [==============================] - 1s 76ms/step - loss: 0.0217 - val_loss: 0.0157
Epoch 952/1000
15/15 [==============================] - 1s 68ms/step - loss: 0.0217 - val_loss: 0.0157
Epoch 953/1000
15/15 [==============================] - 1s 61ms/step - loss: 0.0215 - val_loss: 0.0157
Epoch 954/1000
15/15 [==============================] - 1s 77ms/step - loss: 0.0215 - val_loss: 0.0159
Epoch 955/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.0217 - val_loss: 0.0158
Epoch 956/1000
15/15 [==============================] - 1s 69ms/step - loss: 0.0217 - val_loss: 0.0158
Epoch 957/1000
15/15 [==============================] - 1s 63ms/step - loss: 0.0220 - val_loss: 0.0157
Epoch 958/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.0216 - val_loss: 0.0157
Epoch 959/1000
15/15 [==============================] - 1s 88ms/step - loss: 0.0219 - val_loss: 0.0158
Epoch 960/1000
15/15 [==============================] - 1s 77ms/step - loss: 0.0218 - va

0.01456291414797306

In [37]:

# # Định nghĩa các tham số
# dense_act5 = 'selu'
# recur_act5 = 'sigmoid'
# epoch5 = 1000
# learning_r5 = 0.0005
# ls = 'mae'
# opt = SGD(learning_rate=learning_r5)

# # Encoder
# n_steps_in, n_features = 8, 1  # 8 bước thời gian, 1 đặc trưng
# encoder_inputs = tf.keras.layers.Input(shape=(n_steps_in, n_features))
# encoder_lstm = tf.keras.layers.LSTM(64, activation='tanh', recurrent_activation=recur_act5, return_state=True)
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_states = [state_h, state_c]

# # Decoder
# n_steps_out = 1  # Chúng ta dự đoán một giá trị đầu ra
# decoder_inputs = tf.keras.layers.RepeatVector(n_steps_out)(state_h)
# decoder_lstm = tf.keras.layers.LSTM(64, activation='tanh', recurrent_activation=recur_act5, return_sequences=True)
# decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(32, activation=dense_act5))(decoder_outputs)
# decoder_outputs = tf.keras.layers.Dropout(0.1)(decoder_outputs)
# decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(16, activation=dense_act5))(decoder_outputs)
# decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(8, activation=dense_act5))(decoder_outputs)
# decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder_outputs)

# # Model
# model = tf.keras.models.Model(encoder_inputs, decoder_outputs)
# model.compile(optimizer=opt, loss=ls)

# # Huấn luyện mô hình
# apply4 = model.fit(X_train1, y_train1, epochs=epoch4, batch_size=1024, validation_split=0.25)

# # Đánh giá mô hình
# score4 = model.evaluate(X_te1, y_te1)

# # In ra kết quả
# print(f'Model Score: {score4}')

In [22]:
y_pre1 = model.predict(X_te1)
y_pre2 = model5.predict(X_te2)
# Lưu mô hình và trọng số bằng Keras
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model1_weights.h5")

# Lưu siêu tham số và các thông tin khác bằng joblib
model_info = {
    "model_json_path": "model1.json",
    "model_weights_path": "model1_weights.h5",
    "optimizer": {
        "name": "SGD",
        "learning_rate": learning_r5
    },
    "loss": 'mae'
}

joblib.dump(model_info, 'model1_info.pkl')
# --------------------------------------------------------------
model_json = model5.to_json()
with open("model5.json", "w") as json_file:
    json_file.write(model_json)
model5.save_weights("model5_weights.h5")

# Lưu siêu tham số và các thông tin khác bằng joblib
model_info = {
    "model_json_path": "model1.json",
    "model_weights_path": "model1_weights.h5",
    "optimizer": {
        "name": "SGD",
        "learning_rate": learning_r5
    },
    "loss": 'mae'
}

joblib.dump(model_info, 'model5_info.pkl')

356/356 [==============================] - 1s 4ms/step


['model5_info.pkl']

In [ ]:
'''# Tải lại thông tin mô hình đã lưu
model_info = joblib.load('model1_info.pkl')

# Tải lại mô hình từ JSON và trọng số
with open(model_info['model_json_path'], 'r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(model_info['model_weights_path'])

# Khôi phục optimizer từ thông tin đã lưu
optimizer_info = model_info['optimizer']
if optimizer_info['name'] == 'SGD':
    optimizer = SGD(learning_rate=optimizer_info['learning_rate'])
else:
    raise ValueError(f"Unsupported optimizer: {optimizer_info['name']}")

# Compile lại mô hình với optimizer và loss đã lưu
loaded_model.compile(optimizer=optimizer, loss=model_info['loss'])

# Kiểm tra mô hình đã tải
loaded_model.summary()'''

In [40]:
df_y_pre1 = pd.DataFrame(y_pre1.reshape(-1,1), columns=['y_pre1'], dtype = np.float64)
df_y_pre2 = pd.DataFrame(y_pre2.reshape(-1,1), columns=['y_pre2'], dtype = np.float64)
df_y_real = pd.DataFrame(y_te1.reshape(-1,1), columns=['y_real'])
frames = [df_y_pre1, df_y_pre2, df_y_real]
dataset = pd.concat(frames, axis = 1)
# ------------------------------------------------------------------
X_train_dnn = dataset.drop('y_real', axis =1)[:7000]
y_train_dnn = dataset['y_real'][:7000]
# ------------------------------------
X_te_dnn = dataset.drop('y_real', axis =1)[7000:]
y_te_dnn = dataset['y_real'][7000:]

In [42]:
'''dnn kết hợp kết quả'''
opt_dnn = Adam(learning_rate= 0.01)
model_dnn = Sequential([
                         tf.keras.layers.Dropout(0.1),
                         tf.keras.layers.Dense(32, activation = 'ReLU'),
                         tf.keras.layers.Dropout(0.1),
                         tf.keras.layers.Dense(16, activation = 'ReLU'),
                         tf.keras.layers.Dense(8, activation = 'ReLU'),
                         tf.keras.layers.Dense(1)
                         ])
model_dnn.compile(optimizer = opt_dnn, loss = 'mae', )
apply_dnn = model_dnn.fit(X_train_dnn, y_train_dnn, epochs= epoch4, batch_size =1024, validation_split = 0.25)
score_dnn = model_dnn.evaluate(X_te_dnn, y_te_dnn)
score_dnn

Epoch 1/1000
6/6 [==============================] - 9s 576ms/step - loss: 0.1671 - val_loss: 0.1113
Epoch 2/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.1158 - val_loss: 0.1025
Epoch 3/1000
6/6 [==============================] - 0s 17ms/step - loss: 0.0958 - val_loss: 0.0714
Epoch 4/1000
6/6 [==============================] - 0s 19ms/step - loss: 0.0728 - val_loss: 0.0535
Epoch 5/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0540 - val_loss: 0.0235
Epoch 6/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0508 - val_loss: 0.0528
Epoch 7/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0452 - val_loss: 0.0500
Epoch 8/1000
6/6 [==============================] - 0s 20ms/step - loss: 0.0441 - val_loss: 0.0426
Epoch 9/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0419 - val_loss: 0.0364
Epoch 10/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0417 - val_loss: 0.0332
Epoch 11

6/6 [==============================] - 0s 10ms/step - loss: 0.0301 - val_loss: 0.0613
Epoch 84/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0272 - val_loss: 0.0567
Epoch 85/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0276 - val_loss: 0.0569
Epoch 86/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0260 - val_loss: 0.0564
Epoch 87/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0260 - val_loss: 0.0552
Epoch 88/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0259 - val_loss: 0.0533
Epoch 89/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0261 - val_loss: 0.0581
Epoch 90/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0263 - val_loss: 0.0566
Epoch 91/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0259 - val_loss: 0.0563
Epoch 92/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0259 - val_loss: 0.0538
Epoch 93/1000


6/6 [==============================] - 0s 9ms/step - loss: 0.0244 - val_loss: 0.0615
Epoch 165/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0246 - val_loss: 0.0496
Epoch 166/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0256 - val_loss: 0.0630
Epoch 167/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0267 - val_loss: 0.0512
Epoch 168/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0254 - val_loss: 0.0540
Epoch 169/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0258 - val_loss: 0.0597
Epoch 170/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0267 - val_loss: 0.0533
Epoch 171/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0261 - val_loss: 0.0630
Epoch 172/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0260 - val_loss: 0.0505
Epoch 173/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0258 - val_loss: 0.0581
Epoch 

6/6 [==============================] - 0s 12ms/step - loss: 0.0254 - val_loss: 0.0633
Epoch 246/1000
6/6 [==============================] - 0s 16ms/step - loss: 0.0245 - val_loss: 0.0515
Epoch 247/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0259 - val_loss: 0.0604
Epoch 248/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0244 - val_loss: 0.0638
Epoch 249/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0263 - val_loss: 0.0541
Epoch 250/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0236 - val_loss: 0.0657
Epoch 251/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0244 - val_loss: 0.0545
Epoch 252/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0262 - val_loss: 0.0600
Epoch 253/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0257 - val_loss: 0.0598
Epoch 254/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0240 - val_loss: 0.0571
Epoch

6/6 [==============================] - 0s 14ms/step - loss: 0.0238 - val_loss: 0.0599
Epoch 327/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0230 - val_loss: 0.0595
Epoch 328/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0250 - val_loss: 0.0570
Epoch 329/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0242 - val_loss: 0.0582
Epoch 330/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0236 - val_loss: 0.0621
Epoch 331/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0249 - val_loss: 0.0577
Epoch 332/1000
6/6 [==============================] - 0s 16ms/step - loss: 0.0254 - val_loss: 0.0586
Epoch 333/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0248 - val_loss: 0.0588
Epoch 334/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0251 - val_loss: 0.0580
Epoch 335/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0241 - val_loss: 0.0603
Epoch

...

6/6 [==============================] - 0s 14ms/step - loss: 0.0230 - val_loss: 0.0603
Epoch 732/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0234 - val_loss: 0.0544
Epoch 733/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0238 - val_loss: 0.0616
Epoch 734/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0249 - val_loss: 0.0662
Epoch 735/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0260 - val_loss: 0.0586
Epoch 736/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0244 - val_loss: 0.0577
Epoch 737/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0246 - val_loss: 0.0650
Epoch 738/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0250 - val_loss: 0.0574
Epoch 739/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0250 - val_loss: 0.0561
Epoch 740/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0240 - val_loss: 0.0643
Epoch

6/6 [==============================] - 0s 14ms/step - loss: 0.0249 - val_loss: 0.0604
Epoch 813/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0241 - val_loss: 0.0625
Epoch 814/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0247 - val_loss: 0.0575
Epoch 815/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0244 - val_loss: 0.0587
Epoch 816/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0242 - val_loss: 0.0657
Epoch 817/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0256 - val_loss: 0.0556
Epoch 818/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0250 - val_loss: 0.0613
Epoch 819/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0251 - val_loss: 0.0601
Epoch 820/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0241 - val_loss: 0.0612
Epoch 821/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0253 - val_loss: 0.0577
Epoch

6/6 [==============================] - 0s 15ms/step - loss: 0.0239 - val_loss: 0.0556
Epoch 894/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0254 - val_loss: 0.0619
Epoch 895/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0245 - val_loss: 0.0597
Epoch 896/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0247 - val_loss: 0.0613
Epoch 897/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0252 - val_loss: 0.0626
Epoch 898/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0241 - val_loss: 0.0508
Epoch 899/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0247 - val_loss: 0.0660
Epoch 900/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0241 - val_loss: 0.0577
Epoch 901/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0246 - val_loss: 0.0603
Epoch 902/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0252 - val_loss: 0.0546
Epoch

6/6 [==============================] - 0s 14ms/step - loss: 0.0236 - val_loss: 0.0568
Epoch 975/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0241 - val_loss: 0.0598
Epoch 976/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0243 - val_loss: 0.0604
Epoch 977/1000
6/6 [==============================] - 0s 16ms/step - loss: 0.0251 - val_loss: 0.0575
Epoch 978/1000
6/6 [==============================] - 0s 20ms/step - loss: 0.0231 - val_loss: 0.0570
Epoch 979/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0239 - val_loss: 0.0667
Epoch 980/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0249 - val_loss: 0.0607
Epoch 981/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0236 - val_loss: 0.0539
Epoch 982/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0246 - val_loss: 0.0686
Epoch 983/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0254 - val_loss: 0.0667
Epoch

0.04828362166881561